In [1]:
import os

import pandas as pd

from sklearn.utils import shuffle

from tqdm import tqdm

## 1. Pré-processamento das bases de treino, validação e teste

In [2]:
%%time
path = './datasets'
datasets = ['BC2GM', 'BC4CHEMD', 'BC5CDR-chem', 'BC5CDR-disease', 'JNLPBA', 'linnaeus', 'NCBI-disease', 's800']

train_dataframes = []
valid_dataframes = []
test_dataframes = []

# Itera sobre a lista com os nomes dos diretórios das bases de dado
for data in tqdm(datasets):
    
    # Armazena o caminho do diretória das bases de treino, validação e teste
    train_path = os.path.join(path, data, 'train.tsv')
    valid_path = os.path.join(path, data, 'devel.tsv')
    test_path = os.path.join(path, data, 'test.tsv')

    # Carrega as bases de treino, validação e teste
    train_temp = pd.read_csv(train_path, sep='\t', names=['word', 'tag'], quoting=3)
    valid_temp = pd.read_csv(valid_path, sep='\t', names=['word', 'tag'], quoting=3)
    test_temp = pd.read_csv(test_path, sep='\t', names=['word', 'tag'], quoting=3)

    # Renomeia o padrão IOB de acordo com a tag específica para cada base de dados
    if data == 'BC2GM':
        train_temp['tag'].replace({'I': 'I-DRUG_PROTEIN', 'B': 'B-DRUG_PROTEIN'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-DRUG_PROTEIN', 'B': 'B-DRUG_PROTEIN'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-DRUG_PROTEIN', 'B': 'B-DRUG_PROTEIN'}, inplace=True)

    elif data == 'BC4CHEMD':
        train_temp['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)

    elif data == 'BC5CDR-chem':
        train_temp['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-CHEMICALS', 'B': 'B-CHEMICALS'}, inplace=True)

    elif data == 'BC5CDR-disease':
        train_temp['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)
        
    elif data == 'JNLPBA':
        train_temp['tag'].replace({'I': 'I-DRUG_PROTEIN', 'B': 'B-DRUG_PROTEIN'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-DRUG_PROTEIN', 'B': 'B-DRUG_PROTEIN'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-DRUG_PROTEIN', 'B': 'B-DRUG_PROTEIN'}, inplace=True)

    elif data == 'linnaeus':
        train_temp['tag'].replace({'I': 'I-SPECIES', 'B': 'B-SPECIES'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-SPECIES', 'B': 'B-SPECIES'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-SPECIES', 'B': 'B-SPECIES'}, inplace=True)

    elif data == 'NCBI-disease':
        train_temp['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-DISEASE', 'B': 'B-DISEASE'}, inplace=True)

    elif data == 's800':
        train_temp['tag'].replace({'I': 'I-SPECIES', 'B': 'B-SPECIES'}, inplace=True)
        valid_temp['tag'].replace({'I': 'I-SPECIES', 'B': 'B-SPECIES'}, inplace=True)
        test_temp['tag'].replace({'I': 'I-SPECIES', 'B': 'B-SPECIES'}, inplace=True)

    train_dataframes.append(train_temp)
    valid_dataframes.append(valid_temp)
    test_dataframes.append(test_temp)

100%|██████████| 8/8 [00:01<00:00,  4.72it/s]Wall time: 1.7 s



In [3]:
train = pd.concat(train_dataframes)
valid = pd.concat(valid_dataframes)
test = pd.concat(test_dataframes)

# Sanity check
print(f'A base de treino possui {train.shape[0]:,} linhas e {train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de validação possui {valid.shape[0]:,} linhas e {valid.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de teste possui {test.shape[0]:,} linhas e {test.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 2.493.348 linhas e 2 colunas
A base de validação possui 1.451.029 linhas e 2 colunas
A base de teste possui 1.507.200 linhas e 2 colunas


## 2. Tratando valores faltantes

In [4]:
# Verifica se há missing values na base de treino
print('Valores faltantes na base de treino:' , '-' * 37, train.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de validação
print('Valores faltantes na base de validação:', '-' * 40, valid.isna().sum(), sep='\n', end='\n\n')

# Verifica se há missing values na base de teste
print('Valores faltantes na base de test:', '-' * 35, test.isna().sum(), sep='\n')

Valores faltantes na base de treino:
-------------------------------------
word    220
tag       0
dtype: int64

Valores faltantes na base de validação:
----------------------------------------
word    69
tag      0
dtype: int64

Valores faltantes na base de test:
-----------------------------------
word    79
tag      0
dtype: int64


In [5]:
# Remove dados faltantes da base de treino e validação
train.dropna(inplace=True)
valid.dropna(inplace=True)
test.dropna(inplace=True)

# Sanity check
print(f'A base de treino possui {train.shape[0]:,} linhas e {train.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de validação possui {valid.shape[0]:,} linhas e {valid.shape[1]:,} colunas'.replace(',', '.'))
print(f'A base de teste possui {test.shape[0]:,} linhas e {test.shape[1]:,} colunas'.replace(',', '.'))

A base de treino possui 2.493.128 linhas e 2 colunas
A base de validação possui 1.450.960 linhas e 2 colunas
A base de teste possui 1.507.121 linhas e 2 colunas


## Tratando dados inconsistentes

In [6]:
# Remove dados que impedem a conversão dos arquivos treinamento
pattern = r'\b\t\w*'
mask = train['word'].str.contains(pattern, regex=True)
print('Quantidade de linhas que devem ser removidas na base de treino:', train[mask].shape[0])
train[mask].head()

Quantidade de linhas que devem ser removidas na base de treino: 0


,word,tag


In [7]:
# Remove dados que impedem a conversão dos arquivos validação
pattern = r'\b\t\w*'
mask = valid['word'].str.contains(pattern, regex=True)
print('Quantidade de linhas que devem ser removidas na base de treino:', valid[mask].shape[0])
valid[mask].head()

Quantidade de linhas que devem ser removidas na base de treino: 0


,word,tag


In [8]:
# Remove dados que impedem a conversão dos arquivos de teste
pattern = r'\b\t\w*'
mask = test['word'].str.contains(pattern, regex=True)
print('Quantidade de linhas que devem ser removidas na base de validação:', test[mask].shape[0])
test[mask].head()

Quantidade de linhas que devem ser removidas na base de validação: 0


,word,tag


## Distribuição de _tags_ da variável álvo

In [9]:
train['tag'].value_counts(dropna=False)

O                 2258884
I-DRUG_PROTEIN      86890
B-DRUG_PROTEIN      47375
I-CHEMICALS         37823
B-CHEMICALS         34681
B-DISEASE            9316
I-DISEASE            9033
B-SPECIES            4676
I-SPECIES            4450
Name: tag, dtype: int64

In [10]:
valid['tag'].value_counts(dropna=False)

O                 1335891
I-CHEMICALS         36611
B-CHEMICALS         34833
I-DRUG_PROTEIN      21210
B-DRUG_PROTEIN      11636
B-DISEASE            5033
I-DISEASE            3813
B-SPECIES            1095
I-SPECIES             838
Name: tag, dtype: int64

In [11]:
test['tag'].value_counts(dropna=False)

O                 1397882
I-CHEMICALS         31270
B-CHEMICALS         30724
I-DRUG_PROTEIN      21429
B-DRUG_PROTEIN      12566
B-DISEASE            5384
I-DISEASE            3824
B-SPECIES            2200
I-SPECIES            1842
Name: tag, dtype: int64

In [12]:
# Shuffle data
# train = shuffle(train, random_state=0)

# Salva os arquivos pré-processados em formato "iob"
train.to_csv('./datasets/spacy/train.iob', sep='\t', header=0, index=0)
valid.to_csv('./datasets/spacy/valid.iob', sep='\t', header=0, index=0)
test.to_csv('./datasets/spacy/test.iob', sep='\t', header=0, index=0)